# AWS IoT DR - register a device just-in-time

In the previous notebooks of this series you have created the prerequisites for just-in-time registration with AWS IoT. A topic rule and a Lambda function which finally provisions your device has been created by the CloudFormation stack for the IoT DR solution.

Your device will connect to AWS IoT Core with a certifcate issued by your private CA. Upon the first connect the device will get registered automatically.

## Libraries

In [ ]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient
from os.path import join
import boto3
import json
import logging
import time
import urllib.request

#### Note: If you get an error that the AWSIoTPythonSDK is not installed, install the SDK with the command below and import the libraries again!

In [ ]:
!pip install AWSIoTPythonSDK -t .

## Amazon Root CA

Get the Amazon Root CA which signed the certificate for IoT Core's MQTT message broker. The CA will be used when connecting a device to AWS IoT Core.

In [ ]:
r = urllib.request.urlopen('https://www.amazontrust.com/repository/AmazonRootCA1.pem')
cert_pem = r.read().decode()
print(cert_pem)

f = open('AmazonRootCA1.pem','w')
f.write(cert_pem)
f.close()

## Shared variables

Import shared variables into this notebook.

In [ ]:
%store -r config
print("config: {}".format(json.dumps(config, indent=4, default=str)))

## Boto3 clients

In [ ]:
c_iot_p = boto3.client('iot', region_name = config['aws_region_primary'])
c_iot_s = boto3.client('iot', region_name = config['aws_region_secondary'])

## IoT endpoints

Get the IoT endpoints for AWS IoT Core in the primary and secondary region. In the example below you will connect to the primary region. To try a connection to the secondary region replace the variable `iot_endpoint_primary` with `iot_endpoint_secondary` in the examples below.

In [ ]:
response = c_iot_p.describe_endpoint(endpointType='iot:Data-ATS')
iot_endpoint_primary = response['endpointAddress']
print("iot_endpoint_primary: {}".format(iot_endpoint_primary))

response = c_iot_s.describe_endpoint(endpointType='iot:Data-ATS')
iot_endpoint_secondary = response['endpointAddress']
print("iot_endpoint_secondary: {}".format(iot_endpoint_secondary))

## Connect the device

When a device is reqistered automatically by JITR it will be disconnected automatically after the first connection attempt and the device is being registered. In the first connection attempt you need to provide the device certificate together with the root CA's certificate in one file. After the device has been register you only need to present the device certificate during connect.

After the connection has timed out the code will wait some seconds. It will then configure the credentials to use the device certificate only and connect again to AWS IoT Core.


**Before you connect your device go to the AWS IoT Console -> "MQTT test client" and subscribe to `$aws/events/#` and `cmd/+/pca`.**

When a certificate is being registered automatically AWS IoT Core is publishing a message to the topic

`$aws/events/certificates/registered/[certificateId]`

As you have enable registry events for the solution you will also get messages when a thing is being created. These message are published to the topic

`$aws/events/thing/[clientId]/created`

Set the variable `thing_name` to the same value that you used in the notebook to issue a device certificate.

Feel free to create more certificates and connect more things.

In [ ]:
thing_name = 'dr-pca-04'

root_ca = 'AmazonRootCA1.pem'

device_key_file = '{}.device.key.pem'.format(thing_name)
device_cert_file = '{}.device.cert.pem'.format(thing_name)
device_root_cert_file = '{}.device.root.cert.pem'.format(thing_name)

# AWS IoT Python SDK needs logging
logger = logging.getLogger("AWSIoTPythonSDK.core")
#logger.setLevel(logging.DEBUG)
logger.setLevel(logging.INFO)
streamHandler = logging.StreamHandler()
formatter = logging.Formatter("[%(asctime)s - %(levelname)s - %(filename)s:%(lineno)s - %(funcName)s - %(message)s")
streamHandler.setFormatter(formatter)
logger.addHandler(streamHandler)

myAWSIoTMQTTClient = None
myAWSIoTMQTTClient = AWSIoTMQTTClient(thing_name)
myAWSIoTMQTTClient.configureEndpoint(iot_endpoint_primary, 8883)
myAWSIoTMQTTClient.configureCredentials(root_ca, 
                                        join(config['PCA_directory'], device_key_file), 
                                        join(config['PCA_directory'], device_root_cert_file))

# AWSIoTMQTTClient connection configuration
myAWSIoTMQTTClient.configureAutoReconnectBackoffTime(1, 32, 20)
myAWSIoTMQTTClient.configureOfflinePublishQueueing(-1)  # Infinite offline Publish queueing
myAWSIoTMQTTClient.configureDrainingFrequency(2)  # Draining: 2 Hz
myAWSIoTMQTTClient.configureConnectDisconnectTimeout(10)  # 10 sec
myAWSIoTMQTTClient.configureMQTTOperationTimeout(5)  # 5 sec

# Connect and reconnect to AWS IoT
try:
    myAWSIoTMQTTClient.connect()
except Exception as e:
    logger.info('{}'.format(e))
    myAWSIoTMQTTClient.configureCredentials(root_ca, 
                                        join(config['PCA_directory'], device_key_file), 
                                        join(config['PCA_directory'], device_cert_file))
    time.sleep(5)
    myAWSIoTMQTTClient.connect()

## Verify

Verify that the device has been created in the primary and the secondary region.

### Primary region

In [ ]:
response = c_iot_p.describe_thing(thingName = thing_name)

print("response: {}".format(json.dumps(response, indent=4, default=str)))

### Secondary region

In [ ]:
response = c_iot_s.describe_thing(thingName = thing_name)

print("response: {}".format(json.dumps(response, indent=4, default=str)))

## Publish
Publish a message in the primary region to verify that the device works as expected.

**You have subsribed to "cmd/+/pca# in the primary region?**

In [ ]:
topic = 'cmd/{}/pca'.format(thing_name)
print("topic: {}".format(topic))
message = {"provisioned": "through ACM PCA combined with JITR", "thing_name": "{}".format(thing_name)}

myAWSIoTMQTTClient.publish(topic, json.dumps(message), 0)

Disconnect the device from AWS IoT Core

In [ ]:
myAWSIoTMQTTClient.disconnect()